In [2]:
import pandas as pd
from IPython.display import display, HTML

pd.set_option('display.precision', 3)

In [8]:
def make_vertical(df, score_name):
    if df.shape[1] > df.shape[0]:
        df = df.T
    
    if df.columns.size == 1:
        df.columns = [score_name]
    
    df.index.name = 'Property'
    df_clean = df.dropna()
    return df_clean

In [9]:
df_icc_raw = pd.read_csv('output_icc.csv')
df_icc_vertical = make_vertical(df_icc_raw, 'ICC_Score')

print("--- ICC Results ---")
display(df_icc_vertical)

--- ICC Results ---


,ICC_Score
Property,
Intuition-building,0.243
Clearly Articulated,0.185
Credible,0.832
Scoped,0.458
Impact,-0.039
Strictness,0.158
Actionable,0.526
Discoverable,0.340
Relationships,0.556


In [10]:
df_kappa_raw = pd.read_csv('output_kappa.csv', index_col=0)
df_kappa_vertical = make_vertical(df_kappa_raw, 'Weighted_Kappa')

print("--- Weighted Kappa Results ---")
display(df_kappa_vertical)

--- Weighted Kappa Results ---


,Weighted_Kappa
Property,
Intuition-building,0.337
Clearly Articulated,0.322
Credible,0.746
Scoped,0.383
Impact,-0.053
Strictness,0.235
Actionable,0.562
Discoverable,0.374
Relationships,0.496


In [11]:
comparison = df_icc_vertical.join(df_kappa_vertical, how='outer')
    
print("--- Comparison Table ---")
display(comparison)

--- Comparison Table ---


,ICC_Score,Weighted_Kappa
Property,,
Actionable,0.526,0.562
Clearly Articulated,0.185,0.322
Credible,0.832,0.746
Discoverable,0.340,0.374
Impact,-0.039,-0.053
Intuition-building,0.243,0.337
Permanence,0.535,0.504
Relationships,0.556,0.496
Scoped,0.458,0.383


In [3]:
def get_counts_by_source(filename, coder_name):
    df = pd.read_excel(filename)
    doc_ids = [str(col).strip() for col in df.columns if "Document ID" not in str(col)]
    
    sources = [doc_id[0].upper() for doc_id in doc_ids]
    
    counts = pd.Series(sources).value_counts()
    return counts, set(doc_ids)

sophie_counts, sophie_ids = get_counts_by_source("Validation Study_Sophie.xlsx", "Sophie")
cat_counts, cat_ids = get_counts_by_source("CN_processed.xlsx", "CN")

all_unique_ids = sophie_ids.union(cat_ids)
all_sources = [doc_id[0].upper() for doc_id in all_unique_ids]
total_counts = pd.Series(all_sources).value_counts()

In [4]:
name_map = {
    'V': 'VIS Papers',
    'B': 'Books',
    'C': 'Crowdsourced',
    'W': 'Web Blogs'
}

summary_table = pd.DataFrame({
    'Total Available': total_counts,
    'Sophie Coded': sophie_counts,
    'Cat Coded': cat_counts
})

summary_table = summary_table.fillna(0).astype(int)

summary_table.index = summary_table.index.map(name_map)

desired_order = ['VIS Papers', 'Books', 'Crowdsourced', 'Web Blogs']
existing_order = [x for x in desired_order if x in summary_table.index]
summary_table = summary_table.loc[existing_order]

# Display
from IPython.display import display
print("--- Coding Progress by Source Type ---")
display(summary_table)

--- Coding Progress by Source Type ---


,Total Available,Sophie Coded,Cat Coded
VIS Papers,28,27,27
Books,30,10,30
Crowdsourced,18,16,16
Web Blogs,20,20,0


In [ ]:
def get_ids(filename):
    try:
        df = pd.read_excel(filename)
        return set([str(col).strip() for col in df.columns if "Document ID" not in str(col)])
    except FileNotFoundError:
        return set()

sophie_ids = get_ids("Validation Study_Sophie.xlsx")
cat_ids = get_ids("CN_processed.xlsx")

only_in_sophie = sophie_ids - cat_ids
only_in_cat = cat_ids - sophie_ids

print(f"--- Mismatch Report ---")
print(f"IDs in Sophie's file but NOT in Cat's ({len(only_in_sophie)}):")
print(sorted(list(only_in_sophie)))

print(f"\nIDs in Cat's file but NOT in Sophie's ({len(only_in_cat)}):")
print(sorted(list(only_in_cat)))

--- Mismatch Report ---
IDs in Sophie's file but NOT in Cat's (23):
['CS-10 : VG-GL-12', 'CS-4 : VG-GL-4', 'VIS-6 : VIS-GL-15', 'WB-1 : WB-GL-1', 'WB-10 : WB-GL-10', 'WB-11 : WB-GL-11', 'WB-11 : WB-GL-12', 'WB-11 : WB-GL-13', 'WB-11 : WB-GL-14', 'WB-12 : WB-GL-15', 'WB-13 : WB-GL-16', 'WB-14 : WB-GL-17', 'WB-15 : WB-GL-18', 'WB-16 : WB-GL-19', 'WB-17 : WB-GL-20', 'WB-2 : WB-GL-2', 'WB-3 : WB-GL-3', 'WB-4 : WB-GL-4', 'WB-5 : WB-GL-5', 'WB-6 : WB-GL-6', 'WB-7 : WB-GL-7', 'WB-8 : WB-GL-8', 'WB-9 : WB-GL-9']

IDs in Cat's file but NOT in Sophie's (23):
['BK-2 : BK-GL-11', 'BK-2 : BK-GL-12', 'BK-2 : BK-GL-13', 'BK-2 : BK-GL-14', 'BK-2 : BK-GL-15', 'BK-2 : BK-GL-16', 'BK-2 : BK-GL-17', 'BK-2 : BK-GL-18', 'BK-2 : BK-GL-19', 'BK-3 : BK-GL-20', 'BK-3 : BK-GL-21', 'BK-3 : BK-GL-22', 'BK-3 : BK-GL-23', 'BK-3 : BK-GL-24', 'BK-3 : BK-GL-25', 'BK-3 : BK-GL-26', 'BK-3 : BK-GL-27', 'BK-3 : BK-GL-28', 'BK-3 : BK-GL-29', 'BK-3 : BK-GL-30', 'CS-11 : VG-GL-12', 'CS-4 : VS-GL-4', 'VIS-6: VIS-GL-15']


In [6]:
## Hard Coded for now until we fix that

data = {
    'Total Available': [27, 30, 16, 20],
    'Sophie Coded':    [27, 10, 16, 20],
    'Cat Coded':       [27, 30, 16, 0]
}

index_labels = ['VIS Papers', 'Books', 'Crowdsourced', 'Web Blogs']
summary_table = pd.DataFrame(data, index=index_labels)

print("--- Coding Progress by Source Type ---")
display(summary_table)

--- Coding Progress by Source Type ---


,Total Available,Sophie Coded,Cat Coded
VIS Papers,27,27,27
Books,30,10,30
Crowdsourced,16,16,16
Web Blogs,20,20,0


In [7]:
cat_file_path = "CN_processed.xlsx"
df_cat = pd.read_excel(cat_file_path)

corrections = {
    'CS-11 : VG-GL-12': 'CS-10 : VG-GL-12',
    'CS-4 : VS-GL-4':   'CS-4 : VG-GL-4',
    'VIS-6: VIS-GL-15': 'VIS-6 : VIS-GL-15' 
}

df_cat.rename(columns=corrections, inplace=True)

print("Verifying corrections...")
for old, new in corrections.items():
    if new in df_cat.columns:
        print(f"✅ Success: Found '{new}'")
    else:
        print(f"❌ Warning: Did not find '{new}' (Maybe it was already fixed?)")

new_filename = "CN_processed_FIXED.xlsx"
df_cat.to_excel(new_filename, index=False)
print(f"\nSaved corrected file to: {new_filename}")

Verifying corrections...
✅ Success: Found 'CS-10 : VG-GL-12'
✅ Success: Found 'CS-4 : VG-GL-4'
✅ Success: Found 'VIS-6 : VIS-GL-15'

Saved corrected file to: CN_processed_FIXED.xlsx


In [13]:
df_elements = pd.read_csv('output_elements_kappa.csv', index_col=0)
df_elements.index.name = 'Element (Yes/No)'
df_elements.columns = ['Cohen\'s Kappa']

print("--- Inter-Rater Reliability: Binary Elements ---")

display(df_elements.round(3))

--- Inter-Rater Reliability: Binary Elements ---


,Cohen's Kappa
Element (Yes/No),
Example Present,0.240
Counter-example Present,0.313
Slogan Present,0.481
Action Present,0.411


In [19]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score

# 1. Load Data (Ensure you use the FIXED file for Cat)
r1 = pd.read_excel("Validation Study_Sophie.xlsx")
r2 = pd.read_excel("CN_processed_FIXED.xlsx")

# 2. Find Common Docs
common_docs = r1.columns.intersection(r2.columns)

# 3. Transpose and Clean
def get_clean_df(df, common_cols):
    df_T = df[common_cols].T
    # Use first row as header
    df_T.columns = df_T.iloc[0]
    # Drop header row
    df_T = df_T[1:]
    # Clean column names (strip whitespace)
    df_T.columns = df_T.columns.astype(str).str.strip()
    return df_T

df1 = get_clean_df(r1, common_docs)
df2 = get_clean_df(r2, common_docs)

# 4. Define Targets (Make sure these match your Excel exactly, case-sensitive)
target_cols = [
    "Example Present", 
    "Counter-example Present", 
    "Action Present", 
    "Slogan Present"
]

print(f"{'Property':<30} | {'% Agree':<10} | {'Both Y':<8} | {'Both N':<8} | {'Sophie Y/Cat N':<15} | {'Sophie N/Cat Y':<15}")
print("-" * 110)

found_any = False

for label in target_cols:
    if label in df1.columns:
        found_any = True
        
        # Map Y/N to 1/0 safely
        def safe_map(s):
            return s.astype(str).str.strip().str.upper().map({
                'Y': 1, 'YES': 1, 'N': 0, 'NO': 0
            }).fillna(0)

        s1 = safe_map(df1[label])
        s2 = safe_map(df2[label])
        
        # Calculate Confusion Matrix components
        # 1 = Yes, 0 = No
        both_y = ((s1 == 1) & (s2 == 1)).sum()
        both_n = ((s1 == 0) & (s2 == 0)).sum()
        sophie_y_cat_n = ((s1 == 1) & (s2 == 0)).sum()
        sophie_n_cat_y = ((s1 == 0) & (s2 == 1)).sum()
        
        total = len(s1)
        agreement = (both_y + both_n) / total
        
        print(f"{label:<30} | {agreement:.1%}      | {both_y:<8} | {both_n:<8} | {sophie_y_cat_n:<15} | {sophie_n_cat_y:<15}")

if not found_any:
    print("\nERROR: Still didn't find columns. Here are the available columns in Sophie's file:")
    print(df1.columns.tolist())

Property                       | % Agree    | Both Y   | Both N   | Sophie Y/Cat N  | Sophie N/Cat Y 
--------------------------------------------------------------------------------------------------------------
Example Present                | 60.4%      | 16       | 16       | 16              | 5              
Counter-example Present        | 71.7%      | 7        | 31       | 12              | 3              
Action Present                 | 84.9%      | 41       | 4        | 4               | 4              
Slogan Present                 | 73.6%      | 16       | 23       | 12              | 2              
